In [1]:
# Import required packages
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from sklearn.preprocessing import MinMaxScaler

In [4]:
# Load the cleaned NYPD data
complaints = pd.read_csv("nypd_complaints_cleaned.csv")

In [5]:
# Keep only relevant columns
manhattan_complaints = complaints[["Date", "Crime_Type", "Severity", "Latitude", "Longitude", "severity_weight"]]

In [6]:
# Remove null coordinates
manhattan_complaints = manhattan_complaints.dropna(subset=["Latitude", "Longitude"])

In [7]:
# Convert to GeoDataFrame
gdf = gpd.GeoDataFrame(
    manhattan_complaints,
    geometry=gpd.points_from_xy(manhattan_complaints.Longitude, manhattan_complaints.Latitude),
    crs="EPSG:4326"
)

In [8]:
# Load target zones
zones = gpd.read_file("manhattan_bids.geojson")

In [18]:
# Check if BID IDs need fixing
if zones['bid'].nunique() == 1 and zones['bid'].iloc[0] == 0:
    # Create unique BID IDs
    zones['bid'] = range(1, len(zones) + 1)
    # Save corrected BID file
    zones.to_file("manhattan_bids_fixed.geojson", driver="GeoJSON")

In [19]:
# Create buffered zones for better spatial matching
zones_buffered = zones.copy()
zones_buffered.geometry = zones_buffered.geometry.buffer(0.0005)  # ~50m buffer

C:\Users\lukej\AppData\Local\Temp\ipykernel_25808\3919246293.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  zones_buffered.geometry = zones_buffered.geometry.buffer(0.0005)  # ~50m buffer


In [20]:
# Spatial join to count crimes per zone
joined = gpd.sjoin(gdf, zones_buffered, how="inner", predicate="within")

In [21]:
# Weight by severity and calculate scores
weighted_counts = joined.groupby("bid")["severity_weight"].sum().reset_index(name="weighted_crime")

In [23]:
# Normalise to 1-10 scale (10 = lowest crime)
weighted_counts["crime_score"] = (
    11 - scaler.fit_transform(weighted_counts[["weighted_crime"]])
).round(1)

In [24]:
# Export results
weighted_counts.to_csv("manhattan_crime_scores.csv", index=False)

In [30]:
# Show every crime score for each bid
print(pd.read_csv("manhattan_crime_scores.csv")[['bid', 'crime_score']].to_string(index=False))

 bid  crime_score
   1          6.2
   2          1.0
   3          7.5
   4          6.6
   5          8.4
   6          7.8
   7          1.0
   8          9.1
   9          6.4
  10          7.8
  11          9.8
  12         10.0
  13          3.8
  14          6.9
  15          7.8
  16          4.2
  17          3.0
  18          5.7
  19          9.4
  20          6.6
  21          7.9
  22          9.2
  23          8.3
  24          9.2
  25          8.3
  26          7.7
